# SciViz!

## 2. Using yt

In [ ]:
import yt

We'll use a dataset originally from the yt hub: http://yt-project.org/data/

Specifically, we'll use the IsolatedGalaxy dataset: http://yt-project.org/data/IsolatedGalaxy.tar.gz

Now, lets grab a dataset & upload it.  Here's where mine is stored (in data):

In [ ]:
ds = yt.load("/Users/jnaiman/Downloads/IsolatedGalaxy/galaxy0030/galaxy0030")

Print out various stats of this dataset:

In [ ]:
ds.print_stats()

This is basically telling us something about the number of data points in the dataset. Don't worry if you don't know what levels, grids or cells are at this point we'll get to it later.

Same thing with field list, its cool if some of these look less familiar then others:

In [ ]:
ds.field_list

In [ ]:
ds.derived_field_list

This is a 3D simululation of a galaxy, lets check out some stats about the box:

In [ ]:
ds.domain_right_edge, ds.domain_left_edge

What this is saying is the box goes from (0,0,0) to (1,1,1) in "code_length" units.  Basically, this is just a normalized box.

You can also do fun things like print out max & min densities:

In [ ]:
ds.r[:].max("density"), ds.r[:].min("density")

The above is for the whole box.

We can also ask where the maximum density is in this simulation box:

In [ ]:
ds.r[:].argmax("density")

So this gives us x/y/z positions for where the maximum density is.

Ok, lets make a quick plot 1/2 down the z-direction. 

In [ ]:
# if the plot is too big for class try:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [3, 3]


p = ds.r[:, :, 0.5].plot("density")

Let's zoom:

In [ ]:
p.zoom(10)

So, unless you're an astronomer you might be a little confused about these "kpc" units. But yt allows us to change them!  Behold cool yt units things:

In [ ]:
yt.units.kpc.get_cgs_equivalent()

So we have now changed these weird kpc units.

yt also can do cool things with units like, `yt.units` figures out some math stuff like, making things into cubed cm:

In [ ]:
(yt.units.kpc**3).get_cgs_equivalent()

So let's set some units of our plot!  Let's change the units of density from $g/cm^3$ to $kg/m^3$

In [ ]:
p.set_unit("density","kg/m**3")

We can also include annotations on this plot:

In [ ]:
p.annotate_velocity()

This shows how material is moving in this simulation this is shown with velocity vectors.

We can combine some of our coding around finding max values of density and combine with some region plots.

Let's project the maximum density along the z axis i.e. lets make a plot of the maximum density along the z-axis of our plot:

In [ ]:
p2 = ds.r[:].max("density", axis="z").plot()

We can zoom this as well:

In [ ]:
p2.zoom(10)

If we scroll back up we can see that there is indeed a different between this and our slice plot.  Here, we are much more "smeared" since we're picking only the max density $\rightarrow$ everything looks brighter.

We can also do plots based on region selection but over specific values of z (and x & y). If we recall our box goes from 0$\rightarrow$1 in each x/y/z direction, we can plot a zoom in like so:

In [ ]:
p = ds.r[0.1:0.9, 0.1:0.9, 0.55:0.65].max("density", axis="z").plot()

So, this shows the maximum density but only in a thin slice of the z-axis which is offset from the center.

Since the galaxy lives at the center, and is the highest density gas region, it makes sense that our densities are lower and our features look different -- more "fuzzy ball" outside of the galaxy then gas flowing onto a galaxy disk.

Let's redo the same plot but for the temperature of the gas:

In [ ]:
p = ds.r[0.1:0.9, 0.1:0.9, 0.55:0.65].mean("temperature", axis="z").plot()

We might want to highlight the temperature of the most dense regions.  Why?  Well maybe we want to, instead of depicting the straight temperature, we want to depict the temperature of the *majority of the gas*. We can do this by specifying a "weight" in our projection:

In [ ]:
p = ds.r[0.1:0.9, 0.1:0.9, 0.55:0.65].mean("temperature", weight="density", axis="z").plot()

So why is there this blocky structure?  In space, we don't see cubes around galaxies... yet anyway...

This is becuase this is a simulation of a galaxy, not an actual galaxy.  We can show why this might be by plotting the "grids" of this simulation over this thing:

In [ ]:
p.annotate_grids()

From this we can see that our grids sort of align where the temperature looks funny.  This is a good indicator that we have some numerical artifacts in our simulation.

Ok!  Let's try some more analysis-like plots some of the helpful yt included plots is:

In [ ]:
ds.r[:].profile("density", "temperature").plot()

So this is plotting the temperature of the gas in our simulation, in each binned density.

In our actual simulation, we have temperaturates at a variety of densities, and this is usually the case, so by default what is plotted is the temperature (our 2nd param) plotted at each density bin, but weighted by the mass of material (gas) in each cell.

We can weight by other things, like in this case density:

In [ ]:
ds.r[:].profile("density", "temperature", weight_field="density").plot()

So, similar shape (since mass and density are related) but a little different.

# Activity #2: Brain data with yt

We can also use yt to play with other sorts of data:

In [ ]:
import h5py # might have to pip install

Let's read our datafile into something called "scan_data":

In [ ]:
with h5py.File("/Users/jnaiman/Downloads/single_dicom.h5", "r") as f:
    scan_data = f["/scan"][:]

If we recall, we had a weird shape of this data:

In [ ]:
scan_data.shape

So to import this data into yt to have yt make images for us, we need to do some formatting with numpy:

In [ ]:
import numpy as np

In [ ]:
dsd = yt.load_uniform_grid({'scan': scan_data},
                     [36, 512, 512],
                     #length_unit = yt.units.centimeter, # specify the units of this dataset
                     bbox = np.array([[0., 10], [0, 10], [0, 10]]), # give a "size" to this dataset
)

In [ ]:
dsd.r[:].mean("scan", axis="y").plot(); # this takes the mean along the specified axis "y" and plots

Can also do .max or .min

Note here that the number of fields available is much less:

In [ ]:
dsd.field_list

We can also look at different potions of the z-y axis by specifying the x-axis:

In [ ]:
p = dsd.r[0.75,:,:].plot('scan')

# Activity #3: Output images and objects (3D) with yt
Note: we'll do more with 3D objects next week/the last week, but this is a good first view of some cool ways we can output objects with yt.

Let's go back to to our galaxy object and make a surface.

First, we'll cut down to a sphere and check that out:

In [ ]:
sphere = ds.sphere("max", (500.0, "kpc"))
sphere.mean("density", axis="y").plot(); # this takes the mean along the specified axis "y" and plots

Let's generate a surface of constant density i.e. we'll connect points on a surface where the density has a single value:

In [ ]:
surface = ds.surface(sphere, "density", 1e-27)

In [ ]:
surface.export_obj('/Users/jillnaiman/Downloads/myGalFiles',color_field='temperature')
# the above might take a while

At this point you can upload this to SketchFab, or use PyGEL3D if you were able to install this.

# -----------------------------------------------------------------------

We **WON'T** be doing this from Spring 2021 and on, but its left here as a note for others.

#### If you have PyGEL3D installed:

In [ ]:
# for checking out our surfaces right here
#http://www2.compute.dtu.dk/projects/GEL/PyGEL/
#!pip install PyGEL3D

# you might have to link where pip installs things
# you can find this in your activated DataViz environment with `pip show PyGEL3D`
from sys import path
path.append('/Users/jillnaiman/opt/anaconda3/lib/python3.7/site-packages/')

In [ ]:
from PyGEL3D import gel
from PyGEL3D import js

# for navigating

js.set_export_mode()
m = gel.obj_load("/Users/jillnaiman/Downloads/myGalFiles.obj")
viewer = gel.GLManifoldViewer()
viewer.display(m)



# press ESC to quit?  Yes, but then it takes a while so

In [ ]:
# to get rid of the window
del viewer

Now, lets try with an inline viewer -- also display in the notebook:

In [ ]:
import numpy as np # if you haven't yet

#js.display(m,wireframe=False)
# comment out after you've run since we'll re-run below

Now let's try with an inline viewer & data colors:

In [ ]:
surf_temp = surface['temperature']
surf_temp.shape

We see that this is infact a long list of values temperatures on each surface *face*.

If we look at the shape of the object:

In [ ]:
m.positions().shape, surf_temp.shape[0]*3

We see we have (surf_temp.shape)X3 times the number of points in x/y/z.  This is because these are *vertex* values.  So, if we want to color by something, we should use 3X the number of faces.

In [ ]:
js.display(m, data=np.repeat(np.log10(surf_temp),3),wireframe=False)

We can also process for 3D printing:

In [ ]:
surface.export_obj('/Users/jillnaiman/Downloads/myGalFiles_print',dist_fac=0.001)

## Outputing images for things like clothing

In [ ]:
p = ds.r[:, :, 0.5].plot("density")

In [ ]:
p.zoom(20)

In [ ]:
myImage = p.frb # fixed resoltuion binary

We can then grab a simple image array:

In [ ]:
plt.imshow(np.array(myImage['density']))

... or we can turn off labels and grab a lovely image:

In [ ]:
p = ds.r[:, :, 0.5].plot("density")
p.zoom(10)

In [ ]:
p.hide_colorbar(); p.hide_axes();

In [ ]:
p

Save the image:

In [ ]:
p.save('/Users/jnaiman/Downloads/myImage.png')

Now you have a lovely image that you can upload and put on things like sweaters or whatnot.